In [ ]:
import os
from google.colab import drive

# Mount drive and set the path to your project folder
drive.mount('/content/drive', force_remount=True)
base_path = '/content/drive/MyDrive/deeplearning/'

# List the actual files and folders found at that path
print("--- Actual Contents of Your 'deeplearning' Folder ---")
try:
    actual_contents = os.listdir(base_path)
    for item in actual_contents:
        print(f"'{item}'")
except FileNotFoundError:
    print(f"ERROR: The folder '/content/drive/MyDrive/deeplearning/' itself was not found.")
    print("Please make sure you have a folder named 'deeplearning' in your main My Drive.")

Mounted at /content/drive
--- Actual Contents of Your 'deeplearning' Folder ---
'disease.zip'
'RICE nutri'
'BANANA nutri'
'CoLeaf DATASET'


In [ ]:
import os
import zipfile
from google.colab import drive

# Step 1: Mount your Google Drive
print("Connecting to Google Drive...")
drive.mount('/content/drive', force_remount=True) # Forcing remount to ensure a fresh connection
print("✅ Google Drive connected.")
print("-" * 30)


# Step 2: Define the path to your "deeplearning" folder
base_path = '/content/drive/MyDrive/deeplearning/'


# --- This section is now confirmed with your list ---
banana_path = os.path.join(base_path, 'BANANA nutri')
coleaf_path = os.path.join(base_path, 'CoLeaf DATASET')
rice_path = os.path.join(base_path, 'RICE nutri')
disease_zip_path = os.path.join(base_path, 'disease.zip')
# --- End of section ---


# Step 3: Unzip the disease dataset
disease_unzip_path = '/content/disease_dataset'
if os.path.exists(disease_zip_path):
    print(f"Unzipping {disease_zip_path}...")
    with zipfile.ZipFile(disease_zip_path, 'r') as zip_ref:
        zip_ref.extractall(disease_unzip_path)
    print("✅ Unzipping complete.")
else:
    print(f"❌ ERROR: Cannot find disease.zip at {disease_zip_path}.")
print("-" * 30)


# Step 4: List the internal components of each dataset
dataset_paths = {
    "Disease Dataset": disease_unzip_path,
    "Banana Nutrient": banana_path,
    "CoLeaf Dataset": coleaf_path,
    "Rice Nutrient": rice_path
}

for name, path in dataset_paths.items():
    print(f"🔍 Checking contents of: '{name}'")
    try:
        # List all items in the directory
        all_items = os.listdir(path)
        subfolders = [item for item in all_items if os.path.isdir(os.path.join(path, item))]

        if 'train' in subfolders:
             class_path = os.path.join(path, 'train')
             classes = [d for d in os.listdir(class_path) if os.path.isdir(os.path.join(class_path, d))]
             print(f"  Found {len(classes)} classes inside the 'train' folder.")
             print(f"  Example classes: {classes[:5]}")
        else:
            classes = subfolders
            print(f"  Found {len(classes)} classes.")
            print(f"  Example classes: {classes[:5]}")

    except FileNotFoundError:
        print(f"  ❌ ERROR: Directory not found at '{path}'.")
    print("-" * 30)

Connecting to Google Drive...
Mounted at /content/drive
✅ Google Drive connected.
------------------------------
Unzipping /content/drive/MyDrive/deeplearning/disease.zip...
✅ Unzipping complete.
------------------------------
🔍 Checking contents of: 'Disease Dataset'
  Found 3 classes.
  Example classes: ['new plant diseases dataset(augmented)', 'New Plant Diseases Dataset(Augmented)', 'test']
------------------------------
🔍 Checking contents of: 'Banana Nutrient'
  Found 1 classes.
  Example classes: ['Images of Nutrient Deficient Banana Plant Leaves']
------------------------------
🔍 Checking contents of: 'CoLeaf Dataset'
  Found 1 classes.
  Example classes: ['CoLeaf DATASET']
------------------------------
🔍 Checking contents of: 'Rice Nutrient'
  Found 1 classes.
  Example classes: ['rice_plant_lacks_nutrients']
------------------------------


In [2]:
import tensorflow as tf
import os
import zipfile
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# --- 1. SETUP AND DATA COPY ---
print("TensorFlow Version:", tf.__version__)

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# --- SPEED-UP FIX: COPY DATA FROM DRIVE TO LOCAL DISK ---
print("\nCopying data from Google Drive to local Colab disk...")
print("This may take a few minutes, but will make training much faster.")

# Define source and destination paths
source_drive_path = '/content/drive/MyDrive/deeplearning/'
local_data_path = '/content/local_data/'

# Create the local data directory if it doesn't exist
os.makedirs(local_data_path, exist_ok=True)

# List of folders/files to copy
datasets_to_copy = ['BANANA nutri', 'RICE nutri', 'disease.zip']

# Copy each item
for item in datasets_to_copy:
    source_path = os.path.join(source_drive_path, item)
    destination_path = os.path.join(local_data_path, item)

    if os.path.exists(source_path):
        if os.path.isdir(source_path):
            print(f"Copying folder: {item}...")
            # Remove destination if it exists, to prevent errors on re-run
            if os.path.exists(destination_path):
                shutil.rmtree(destination_path)
            shutil.copytree(source_path, destination_path)
        else: # It's a file
            print(f"Copying file: {item}...")
            shutil.copy(source_path, destination_path)
    else:
        print(f"WARNING: Could not find {source_path}")

print("\n✅ Data copy complete.")
# --- END OF SPEED-UP FIX ---


# --- NOW, USE THE FAST LOCAL PATHS FOR EVERYTHING ---
base_path = local_data_path # <-- IMPORTANT: Use the local path from now on

# Unzip the main disease dataset from its new local location
disease_zip_path = os.path.join(base_path, 'disease.zip')
disease_unzip_path = '/content/disease_dataset' # Unzip to a different temp folder
print(f"\nUnzipping {disease_zip_path}...")
with zipfile.ZipFile(disease_zip_path, 'r') as zip_ref:
    zip_ref.extractall(disease_unzip_path)
print("Unzipping complete.")

# --- AUTOMATIC PATH CORRECTION FOR DISEASE DATASET ---
disease_train_path = ''
for root, dirs, files in os.walk(disease_unzip_path):
    if 'train' in dirs:
        disease_train_path = os.path.join(root, 'train')
        break
if disease_train_path:
    print(f"✅ Automatically found disease train path at: {disease_train_path}")
else:
    raise FileNotFoundError("Could not automatically find a 'train' directory inside the unzipped disease dataset.")

# Define paths to all datasets using the local base_path
paths = {
    'disease': disease_train_path,
    'banana': os.path.join(base_path, 'BANANA nutri'),
    'rice': os.path.join(base_path, 'RICE nutri')
}


# --- 2. DATA PREPARATION (ROBUST VERSION 2.0) ---
# (This section remains the same, it will now just run faster)
print("\nPreparing data...")
image_paths = []
disease_labels_str = []
nutrient_labels_str = []
valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

# Step A: Find all images and their string labels first
for class_name in os.listdir(paths['disease']):
    class_path = os.path.join(paths['disease'], class_name)
    if not os.path.isdir(class_path): continue
    for img_name in os.listdir(class_path):
        if any(img_name.lower().endswith(ext) for ext in valid_extensions):
            image_paths.append(os.path.join(class_path, img_name))
            disease_labels_str.append(class_name)
            nutrient_labels_str.append('not_applicable')

for prefix, path in [('banana', paths['banana']), ('rice', paths['rice'])]:
    for root, dirs, files in os.walk(path):
        if files and not dirs:
            for img_name in files:
                if any(img_name.lower().endswith(ext) for ext in valid_extensions):
                    class_name = os.path.basename(root)
                    image_paths.append(os.path.join(root, img_name))
                    disease_labels_str.append('healthy_plant')
                    nutrient_labels_str.append(f"{prefix}_{class_name}")

print(f"Total images found: {len(image_paths)}")

# Step B: Create class lists and maps from the collected string labels
disease_classes = sorted(list(set(disease_labels_str)))
nutrient_classes = sorted(list(set(nutrient_labels_str)))
disease_map = {name: i for i, name in enumerate(disease_classes)}
nutrient_map = {name: i for i, name in enumerate(nutrient_classes)}

print(f"Found {len(disease_classes)} disease classes.")
print(f"Found {len(nutrient_classes)} nutrient classes.")

# Step C: Convert string labels to integer indices
disease_labels_int = [disease_map[label] for label in disease_labels_str]
nutrient_labels_int = [nutrient_map[label] for label in nutrient_labels_str]

# Split data into training and validation sets
X_train, X_val, y_disease_train, y_disease_val, y_nutrient_train, y_nutrient_val = train_test_split(
    image_paths, disease_labels_int, nutrient_labels_int, test_size=0.2, random_state=42, stratify=disease_labels_int
)

# One-hot encode the labels
y_disease_train = to_categorical(y_disease_train, num_classes=len(disease_classes))
y_disease_val = to_categorical(y_disease_val, num_classes=len(disease_classes))
y_nutrient_train = to_categorical(y_nutrient_train, num_classes=len(nutrient_classes))
y_nutrient_val = to_categorical(y_nutrient_val, num_classes=len(nutrient_classes))


# --- 3. TF.DATA PIPELINE ---
IMG_SIZE = 224
BATCH_SIZE = 32

def load_and_preprocess(path, disease_label, nutrient_label):
    image = tf.io.read_file(path)
    image = tf.image.decode_image(image, channels=3, expand_animations=False)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image, ({"disease_output": disease_label, "nutrient_output": nutrient_label})

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_disease_train, y_nutrient_train))
train_ds = train_ds.shuffle(buffer_size=len(X_train)).map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_disease_val, y_nutrient_val))
val_ds = val_ds.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


# --- 4. MODEL BUILDING ---
print("\nBuilding model...")
base_model = tf.keras.applications.EfficientNetV2B0(
    include_top=False,
    weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)

disease_head = tf.keras.layers.Dense(len(disease_classes), activation='softmax', name='disease_output')(x)
nutrient_head = tf.keras.layers.Dense(len(nutrient_classes), activation='softmax', name='nutrient_output')(x)

model = tf.keras.Model(inputs=inputs, outputs=[disease_head, nutrient_head])

# --- 5. MODEL COMPILATION ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss={
        'disease_output': 'categorical_crossentropy',
        'nutrient_output': 'categorical_crossentropy'
    },
    metrics={
        'disease_output': 'accuracy',
        'nutrient_output': 'accuracy'
    }
)
model.summary()

# --- 6. TRAINING ---
print("\nStarting training...")
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model.weights.h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True,
    mode='min'
)

EPOCHS = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint]
)

# --- 7. SAVING THE FINAL MODEL ---
print("\nTraining complete. Saving the final model...")
model.load_weights('best_model.weights.h5')
model.save('plant_model_final.h5')

print("\n✅ Successfully saved 'plant_model_final.h5'. You can now download this file for use in VS Code.")
np.save('disease_classes.npy', np.array(disease_classes))
np.save('nutrient_classes.npy', np.array(nutrient_classes))
print("✅ Saved class lists to 'disease_classes.npy' and 'nutrient_classes.npy'. Download these as well.")

TensorFlow Version: 2.19.0
Mounted at /content/drive

Copying data from Google Drive to local Colab disk...
This may take a few minutes, but will make training much faster.
Copying folder: BANANA nutri...
Copying folder: RICE nutri...
Copying file: disease.zip...

✅ Data copy complete.

Unzipping /content/local_data/disease.zip...
Unzipping complete.
✅ Automatically found disease train path at: /content/disease_dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train

Preparing data...
Total images found: 82051
Found 39 disease classes.
Found 13 nutrient classes.

Building model...


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficientnetv2-b0   │ (None, 7, 7,      │  5,919,312 │ input_layer_3[0]… │
│ (Functional)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ efficientnetv2-b… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 1280)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ disease_output      │ (None, 39)        │     49,959 │ dropout_1[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ nutrient_output     │ (None, 13)        │     16,653 │ dropout_1[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,985,924 (22.83 MB)

 Trainable params: 66,612 (260.20 KB)

 Non-trainable params: 5,919,312 (22.58 MB)


Starting training...
Epoch 1/10
2052/2052 ━━━━━━━━━━━━━━━━━━━━ 203s 85ms/step - disease_output_accuracy: 0.7905 - disease_output_loss: 0.9170 - loss: 1.2567 - nutrient_output_accuracy: 0.8949 - nutrient_output_loss: 0.3398 - val_disease_output_accuracy: 0.9629 - val_disease_output_loss: 0.1548 - val_loss: 0.3170 - val_nutrient_output_accuracy: 0.9416 - val_nutrient_output_loss: 0.1621
Epoch 2/10
2052/2052 ━━━━━━━━━━━━━━━━━━━━ 124s 60ms/step - disease_output_accuracy: 0.9494 - disease_output_loss: 0.1850 - loss: 0.3535 - nutrient_output_accuracy: 0.9399 - nutrient_output_loss: 0.1686 - val_disease_output_accuracy: 0.9763 - val_disease_output_loss: 0.0948 - val_loss: 0.2429 - val_nutrient_output_accuracy: 0.9456 - val_nutrient_output_loss: 0.1481
Epoch 3/10
2052/2052 ━━━━━━━━━━━━━━━━━━━━ 136s 57ms/step - disease_output_accuracy: 0.9600 - disease_output_loss: 0.1350 - loss: 0.2869 - nutrient_output_accuracy: 0.9454 - nutrient_output_loss: 0.1519 - val_disease_output_accuracy: 0.9784 - va


✅ Successfully saved 'plant_model_final.h5'. You can now download this file for use in VS Code.
✅ Saved class lists to 'disease_classes.npy' and 'nutrient_classes.npy'. Download these as well.


In [3]:
import tensorflow as tf
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# --- 1. SETUP AND PATHS ---
print("TensorFlow Version:", tf.__version__)

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define the path to your RICE nutri folder
rice_path = '/content/drive/MyDrive/deeplearning/RICE nutri/'

# --- 2. DATA PREPARATION (RICE ONLY) ---
print("\nPreparing rice data...")
image_paths = []
labels_str = []
valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

# Recursively find all images and their class labels
for root, dirs, files in os.walk(rice_path):
    if files and not dirs: # Only process leaf directories that contain files
        for img_name in files:
            if any(img_name.lower().endswith(ext) for ext in valid_extensions):
                class_name = os.path.basename(root)
                image_paths.append(os.path.join(root, img_name))
                labels_str.append(class_name)

print(f"Total images found: {len(image_paths)}")

# Create the class list and map
rice_classes = sorted(list(set(labels_str)))
class_map = {name: i for i, name in enumerate(rice_classes)}
labels_int = [class_map[label] for label in labels_str]

print(f"Found {len(rice_classes)} rice nutrient classes: {rice_classes}")

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    image_paths, labels_int, test_size=0.2, random_state=42, stratify=labels_int
)

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=len(rice_classes))
y_val = to_categorical(y_val, num_classes=len(rice_classes))

# --- 3. TF.DATA PIPELINE ---
IMG_SIZE = 224
BATCH_SIZE = 32

def load_and_preprocess(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_image(image, channels=3, expand_animations=False)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(buffer_size=len(X_train)).map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_ds = val_ds.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# --- 4. MODEL BUILDING (SINGLE OUTPUT) ---
print("\nBuilding model...")
base_model = tf.keras.applications.EfficientNetV2B0(
    include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(len(rice_classes), activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

# --- 5. MODEL COMPILATION ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

# --- 6. TRAINING ---
print("\nStarting training...")
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'rice_best_model.weights.h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True,
    mode='min'
)

EPOCHS = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint]
)

# --- 7. SAVING THE FINAL MODEL ---
print("\nTraining complete. Saving the final rice model...")
model.load_weights('rice_best_model.weights.h5')
model.save('rice_model_final.h5')

print("\n✅ Successfully saved 'rice_model_final.h5'.")
np.save('rice_classes.npy', np.array(rice_classes))
print("✅ Saved class list to 'rice_classes.npy'.")

TensorFlow Version: 2.19.0
Mounted at /content/drive

Preparing rice data...
Total images found: 1156
Found 3 rice nutrient classes: ['Nitrogen(N)', 'Phosphorus(P)', 'Potassium(K)']

Building model...


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b0 (Functional)  │ (None, 7, 7, 1280)     │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │         3,843 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,923,155 (22.60 MB)

 Trainable params: 3,843 (15.01 KB)

 Non-trainable params: 5,919,312 (22.58 MB)


Starting training...
Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.4788 - loss: 1.0268 - val_accuracy: 0.8060 - val_loss: 0.6241
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 23s 792ms/step - accuracy: 0.7771 - loss: 0.5854 - val_accuracy: 0.8793 - val_loss: 0.4440
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 25s 874ms/step - accuracy: 0.8844 - loss: 0.4064 - val_accuracy: 0.8836 - val_loss: 0.3714
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 24s 822ms/step - accuracy: 0.8848 - loss: 0.3621 - val_accuracy: 0.9052 - val_loss: 0.3243
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 41s 828ms/step - accuracy: 0.8999 - loss: 0.3118 - val_accuracy: 0.9224 - val_loss: 0.2946
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 42s 873ms/step - accuracy: 0.9247 - loss: 0.2789 - val_accuracy: 0.9181 - val_loss: 0.2760
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 40s 825ms/step - accuracy: 0.9046 - loss: 0.2738 - val_accuracy: 0.9224 - val_loss: 0.2610
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 39s 761ms/step - accuracy: 0.9350 - loss


✅ Successfully saved 'rice_model_final.h5'.
✅ Saved class list to 'rice_classes.npy'.


In [4]:
import tensorflow as tf
import os
import zipfile
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# --- 1. SETUP AND PATHS ---
print("TensorFlow Version:", tf.__version__)

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define the path to your deeplearning folder
base_path = '/content/drive/MyDrive/deeplearning/'

# Unzip the disease dataset
disease_zip_path = os.path.join(base_path, 'disease.zip')
disease_unzip_path = '/content/disease_dataset'
print(f"Unzipping {disease_zip_path}...")
with zipfile.ZipFile(disease_zip_path, 'r') as zip_ref:
    zip_ref.extractall(disease_unzip_path)
print("Unzipping complete.")

# --- AUTOMATIC PATH CORRECTION FOR DISEASE DATASET ---
disease_train_path = ''
for root, dirs, files in os.walk(disease_unzip_path):
    if 'train' in dirs:
        disease_train_path = os.path.join(root, 'train')
        break
if disease_train_path:
    print(f"✅ Automatically found disease train path at: {disease_train_path}")
else:
    raise FileNotFoundError("Could not automatically find a 'train' directory inside the unzipped disease dataset.")


# --- 2. DATA PREPARATION (DISEASE ONLY) ---
print("\nPreparing disease data...")
image_paths = []
labels_str = []
valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

# Recursively find all images and their class labels
for class_name in os.listdir(disease_train_path):
    class_path = os.path.join(disease_train_path, class_name)
    if not os.path.isdir(class_path): continue
    for img_name in os.listdir(class_path):
        if any(img_name.lower().endswith(ext) for ext in valid_extensions):
            image_paths.append(os.path.join(class_path, img_name))
            labels_str.append(class_name)

print(f"Total images found: {len(image_paths)}")

# Create the class list and map
disease_classes = sorted(list(set(labels_str)))
class_map = {name: i for i, name in enumerate(disease_classes)}
labels_int = [class_map[label] for label in labels_str]

print(f"Found {len(disease_classes)} disease classes.")

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    image_paths, labels_int, test_size=0.2, random_state=42, stratify=labels_int
)

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=len(disease_classes))
y_val = to_categorical(y_val, num_classes=len(disease_classes))

# --- 3. TF.DATA PIPELINE ---
IMG_SIZE = 224
BATCH_SIZE = 32

def load_and_preprocess(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_image(image, channels=3, expand_animations=False)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(buffer_size=len(X_train)).map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_ds = val_ds.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# --- 4. MODEL BUILDING (SINGLE OUTPUT) ---
print("\nBuilding model...")
base_model = tf.keras.applications.EfficientNetV2B0(
    include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(len(disease_classes), activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

# --- 5. MODEL COMPILATION ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

# --- 6. TRAINING ---
print("\nStarting training...")
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'disease_best_model.weights.h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True,
    mode='min'
)

EPOCHS = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint]
)

# --- 7. SAVING THE FINAL MODEL ---
print("\nTraining complete. Saving the final disease model...")
model.load_weights('disease_best_model.weights.h5')
model.save('disease_model_final.h5')

print("\n✅ Successfully saved 'disease_model_final.h5'.")
np.save('disease_classes.npy', np.array(disease_classes))
print("✅ Saved class list to 'disease_classes.npy'.")

TensorFlow Version: 2.19.0
Mounted at /content/drive
Unzipping /content/drive/MyDrive/deeplearning/disease.zip...
Unzipping complete.
✅ Automatically found disease train path at: /content/disease_dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train

Preparing disease data...
Total images found: 70295
Found 38 disease classes.

Building model...


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b0 (Functional)  │ (None, 7, 7, 1280)     │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 38)             │        48,678 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,967,990 (22.77 MB)

 Trainable params: 48,678 (190.15 KB)

 Non-trainable params: 5,919,312 (22.58 MB)


Starting training...
Epoch 1/10
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 173s 84ms/step - accuracy: 0.7538 - loss: 1.0855 - val_accuracy: 0.9570 - val_loss: 0.1862
Epoch 2/10
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 85s 48ms/step - accuracy: 0.9436 - loss: 0.2168 - val_accuracy: 0.9714 - val_loss: 0.1164
Epoch 3/10
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 141s 48ms/step - accuracy: 0.9534 - loss: 0.1589 - val_accuracy: 0.9757 - val_loss: 0.0921
Epoch 4/10
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 90s 51ms/step - accuracy: 0.9577 - loss: 0.1396 - val_accuracy: 0.9783 - val_loss: 0.0804
Epoch 5/10
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 142s 51ms/step - accuracy: 0.9620 - loss: 0.1249 - val_accuracy: 0.9776 - val_loss: 0.0738
Epoch 6/10
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 137s 49ms/step - accuracy: 0.9648 - loss: 0.1165 - val_accuracy: 0.9816 - val_loss: 0.0668
Epoch 7/10
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 90s 51ms/step - accuracy: 0.9650 - loss: 0.1107 - val_accuracy: 0.9809 - val_loss: 0.0639
Epoch 8/10
1758/1758 ━━━━━━━━━━━━━━━━━━━━ 138s 49m


✅ Successfully saved 'disease_model_final.h5'.
✅ Saved class list to 'disease_classes.npy'.
